The dataset can be downloaded from here and it consists of 18000 grayscale images (18000x150x150
or 18000x75x75) contained in ‘images.npy’. The labels for each sample are represented by two integers
(18000x2, ‘labels.npy’ file), that correspond to the hour and minute displayed by the clock. You can see
that each image is rendered from a different angle and rotation and they might contain light reflections from
within the scene making this a non-trivial problem. For your experiments, we suggest splitting your data
into 80/10/10% splits for training/validation and test sets respectively. Remember to shuffle your dataset
as the sample files are ordered. We suggest using the smaller dataset for your initial tests and runs (75x75
images) and then reporting your results on the larger (150x150) datase

## GPU CHECK

In [1]:
import tensorflow  as tf
print(tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"TensorFlow detected {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"  - {gpu.name}")
else:
    print("TensorFlow did NOT detect any GPUs. It will use the CPU.")


2025-11-02 13:31:39.200800: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-02 13:31:39.236685: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-02 13:31:40.090622: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2.20.0
TensorFlow detected 1 GPU(s):
  - /physical_device:GPU:0


#### import needed packages

In [2]:

# from tensorflow import keras
import os
# import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
import keras


#### load data

In [3]:
pixel_size = 150
data_folder = f"A1_data_{pixel_size}"
images_path = os.path.join(data_folder, "images.npy")
images = np.load(images_path)
labels_path = os.path.join(data_folder, "labels.npy")
labels = np.load(labels_path)


(a) Classification - treat this as a n-class classification problem. We suggest starting out with a
smaller number of categories e.g. grouping all the samples that are between [3 : 00 −3 : 30] into
a single category (results in 24 categories in total), and trying to train a CNN model. Once you
have found a working architecture, increase the number of categories by using smaller intervals
for grouping samples to increase the ’common sense accuracy’. Can you train a network using
all 720 different labels? What problems does such a label representation have?

## Task a: classification
We will start with deviding labels into 24 categories, one for each 30 minute

In [4]:

print(labels)
def get_cat_labels(labels):
    new_labels = []
    for label in labels:
        label = label[0]* 2 + int(label[1] >= 30)
        new_labels.append(label)
    return np.array(new_labels)
labels = get_cat_labels(labels)
print(labels)


[[ 0  0]
 [ 0  0]
 [ 0  0]
 ...
 [11 59]
 [11 59]
 [11 59]]
[ 0  0  0 ... 23 23 23]


We then split the data into training, validation, and test sets. The sklearn train_test_split method shuffles the data by default

In [5]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1.


we define a common sense loss. This will calculate how far of the prediction was

In [6]:
def make_common_sense_loss(num_classes=24.0):
    def common_sense_loss(y_true, y_pred):
        y_pred_class = tf.argmax(y_pred, axis=1)
        y_true_float = tf.cast(tf.squeeze(y_true), dtype=tf.float32)
        y_pred_float = tf.cast(y_pred_class, dtype=tf.float32)
        diff = tf.abs(y_true_float - y_pred_float)
        cyclical_diff = tf.minimum(diff, num_classes- diff)
        return tf.reduce_mean(cyclical_diff * 720 /num_classes)
    return common_sense_loss

Our model for 24 class classification. We use a scheduler to lower the learning rate when we plateau



In [7]:
def create_model(num_classes=24):
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,          # halve the learning rate if there is no improvement
        patience=3,          # Wait 2 epochs with no improvement before reducing
        min_lr=1e-6          # Set a minimum learning rate at 1e-6
    )
    early_stopper = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6,          # Wait 6 epochs for improvement before stopping
        restore_best_weights=True  # Automatically restore the weights from the best epoch
    )
    model= keras.models.Sequential([
        keras.Input(shape=(pixel_size, pixel_size, 1)),
        # Block 1
        keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2,2)),
        # keras.layers.Dropout(0.2),
        # Block 2
        keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        # keras.layers.Dropout(0.2),
        # Block 3
        keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        # keras.layers.Dropout(0.2),
        # Block 4
        keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        # keras.layers.Dropout(0.2), #dropout not needed, bacuase of max pooling

        # #block 5
        # keras.layers.Conv2D(512, (3,3), activation="relu", padding="same"),
        # keras.layers.BatchNormalization(),
        # keras.layers.Conv2D(512, (3,3), activation="relu", padding="same"),
        # keras.layers.BatchNormalization(),
        # keras.layers.MaxPooling2D(2),
        # # keras.layers.Dropout(0.2),

        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="leaky_relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="leaky_relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(int(num_classes), activation="softmax")
    ])
    cse = make_common_sense_loss(num_classes=num_classes)
    model.compile(loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=[cse,"Accuracy"
            #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
            ],
    )
    return model, lr_scheduler, early_stopper
model, lr_scheduler, early_stopper = create_model()


I0000 00:00:1762086701.212115  536960 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6119 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [8]:
model.save("time_prediction_model_class.keras")

In [18]:
# #make plot for model architecture
# keras.utils.plot_model(
#     model,
#     to_file="model_plot_compact.png",
#     show_shapes=True,
#     show_layer_names=False,
#     show_layer_activations=False
# )

In [10]:
# model.fit(
#     X_train, y_train,
#     epochs=60,
#     validation_data=(X_valid, y_valid),
#     callbacks=[lr_scheduler, early_stopper]
#     )
# #evaluate the model on the test set
# test_loss,test_csl, test_acc = model.evaluate(X_test, y_test)
# print('Test accuracy:', test_acc)
# #base:0.8420000076293945
# #leaky: 0.8525000214576721
# #leaky + L2regularization: 0.8472999930381775
# #leaky + batch normalization: 0.8978000283241272
# #0.9711111187934875
# #0.36 at 25
# # metrics=["accuracy"])

We now make a class for every 10 minutes

In [11]:
labels = np.load(labels_path)

def get_cat_labels_10(labels):
    new_labels = []
    dct = {}
    for label in labels:
        old = label
        label = label[0]* 6 + int((label[1])/10)
        new_labels.append(label)
        dct[str(old)] = label
    print(dct)
    return np.array(new_labels)
labels = get_cat_labels_10(labels)
# labels = keras.utils.to_categorical(labels, num_classes=72)
print(labels)


{'[0 0]': np.int64(0), '[0 1]': np.int64(0), '[0 2]': np.int64(0), '[0 3]': np.int64(0), '[0 4]': np.int64(0), '[0 5]': np.int64(0), '[0 6]': np.int64(0), '[0 7]': np.int64(0), '[0 8]': np.int64(0), '[0 9]': np.int64(0), '[ 0 10]': np.int64(1), '[ 0 11]': np.int64(1), '[ 0 12]': np.int64(1), '[ 0 13]': np.int64(1), '[ 0 14]': np.int64(1), '[ 0 15]': np.int64(1), '[ 0 16]': np.int64(1), '[ 0 17]': np.int64(1), '[ 0 18]': np.int64(1), '[ 0 19]': np.int64(1), '[ 0 20]': np.int64(2), '[ 0 21]': np.int64(2), '[ 0 22]': np.int64(2), '[ 0 23]': np.int64(2), '[ 0 24]': np.int64(2), '[ 0 25]': np.int64(2), '[ 0 26]': np.int64(2), '[ 0 27]': np.int64(2), '[ 0 28]': np.int64(2), '[ 0 29]': np.int64(2), '[ 0 30]': np.int64(3), '[ 0 31]': np.int64(3), '[ 0 32]': np.int64(3), '[ 0 33]': np.int64(3), '[ 0 34]': np.int64(3), '[ 0 35]': np.int64(3), '[ 0 36]': np.int64(3), '[ 0 37]': np.int64(3), '[ 0 38]': np.int64(3), '[ 0 39]': np.int64(3), '[ 0 40]': np.int64(4), '[ 0 41]': np.int64(4), '[ 0 42]': 

In [12]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default
print(y_valid)


[48 33 45 ... 41  4  0]


In [13]:
model, lr_scheduler, early_stopper = create_model(num_classes=72.0)

In [14]:
# print(X_valid.shape)
# model.fit(
#     X_train, y_train,
#     epochs=60,
#     validation_data=(X_valid, y_valid),
#     callbacks=[lr_scheduler, early_stopper]
#     )
# #evaluate the model on the test set
# test_loss,test_csl, test_acc = model.evaluate(X_test, y_test)
# print('Test accuracy:', test_acc)
# #base:0.8420000076293945
# #leaky: 0.8525000214576721
# #leaky + L2regularization: 0.8472999930381775
# #leaky + batch normalization: 0.8978000283241272

# #0.9194444417953491
# # metrics=["accuracy"])

In [36]:
labels = np.load(labels_path)
def get_cat_labels_min(labels):
    new_labels = []
    dct = {}
    for label in labels:
        old = label
        label = label[0]* 60 + int((label[1]))
        new_labels.append(label)
        dct[str(old)] = label
    print(dct)
    return np.array(new_labels)
labels = get_cat_labels_min(labels)


{'[0 0]': np.int64(0), '[0 1]': np.int64(1), '[0 2]': np.int64(2), '[0 3]': np.int64(3), '[0 4]': np.int64(4), '[0 5]': np.int64(5), '[0 6]': np.int64(6), '[0 7]': np.int64(7), '[0 8]': np.int64(8), '[0 9]': np.int64(9), '[ 0 10]': np.int64(10), '[ 0 11]': np.int64(11), '[ 0 12]': np.int64(12), '[ 0 13]': np.int64(13), '[ 0 14]': np.int64(14), '[ 0 15]': np.int64(15), '[ 0 16]': np.int64(16), '[ 0 17]': np.int64(17), '[ 0 18]': np.int64(18), '[ 0 19]': np.int64(19), '[ 0 20]': np.int64(20), '[ 0 21]': np.int64(21), '[ 0 22]': np.int64(22), '[ 0 23]': np.int64(23), '[ 0 24]': np.int64(24), '[ 0 25]': np.int64(25), '[ 0 26]': np.int64(26), '[ 0 27]': np.int64(27), '[ 0 28]': np.int64(28), '[ 0 29]': np.int64(29), '[ 0 30]': np.int64(30), '[ 0 31]': np.int64(31), '[ 0 32]': np.int64(32), '[ 0 33]': np.int64(33), '[ 0 34]': np.int64(34), '[ 0 35]': np.int64(35), '[ 0 36]': np.int64(36), '[ 0 37]': np.int64(37), '[ 0 38]': np.int64(38), '[ 0 39]': np.int64(39), '[ 0 40]': np.int64(40), '[ 0

In [39]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default
# get the number of instances in each class
unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))
print("Class distribution in y_train:")
for class_label, count in class_counts.items():
    if count < 17 or count > 24:
        print(f"Class {class_label}: {count} instances")

Class distribution in y_train:
Class 1: 12 instances
Class 27: 15 instances
Class 40: 25 instances
Class 64: 14 instances
Class 66: 16 instances
Class 109: 16 instances
Class 156: 16 instances
Class 161: 16 instances
Class 162: 15 instances
Class 297: 16 instances
Class 312: 16 instances
Class 316: 15 instances
Class 320: 16 instances
Class 367: 16 instances
Class 370: 16 instances
Class 376: 15 instances
Class 377: 16 instances
Class 383: 15 instances
Class 384: 15 instances
Class 412: 25 instances
Class 420: 16 instances
Class 473: 25 instances
Class 475: 25 instances
Class 493: 16 instances
Class 514: 16 instances
Class 576: 16 instances
Class 596: 16 instances
Class 600: 15 instances
Class 609: 16 instances
Class 620: 16 instances
Class 633: 16 instances
Class 647: 16 instances
Class 649: 16 instances
Class 717: 14 instances


In [ ]:
model, lr_scheduler, early_stopper = create_model(num_classes=720)

In [ ]:
# model.fit(
#     X_train, y_train,
#     epochs=60,
#     validation_data=(X_valid, y_valid),
#     callbacks=[lr_scheduler, early_stopper]
#     )
# # evaluate the model on the test set
# test_loss,test_csl, test_acc = model.evaluate(X_test, y_test)

# print('Test accuracy:', test_acc)
# (print(tf.__version__))
# #0.9194444417953491
# # metrics=["accuracy"])

In [ ]:
#feature prep for dual head

labels = np.load(labels_path)


In [ ]:
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default

In [ ]:
y_train_hours = y_train[:, 0]
y_train_minutes = y_train[:, 1]
y_valid_hours = y_valid[:, 0]
y_valid_minutes = y_valid[:, 1]
y_test_hours = y_test[:, 0]
y_test_minutes = y_test[:, 1]

#one-hot encoding the labels
y_train_hours_enc = keras.utils.to_categorical(y_train_hours, num_classes=12)
y_train_minutes_enc = y_train_minutes
y_valid_hours_enc = keras.utils.to_categorical(y_valid_hours, num_classes=12)
y_valid_minutes_enc = y_valid_minutes
y_test_hours_enc = keras.utils.to_categorical(y_test_hours, num_classes=12)
y_test_minutes_enc = y_test_minutes
y_train_formatted = {
    "hour_output": y_train_hours_enc,
    "minute_output": y_train_minutes_enc
}

y_valid_formatted = {
    "hour_output": y_valid_hours_enc,
    "minute_output": y_valid_minutes_enc
}
y_train_formatted = {
    "hour_output": y_train_hours_enc,
    "minute_output": y_train_minutes_enc
}

y_valid_formatted = {
    "hour_output": y_valid_hours_enc,
    "minute_output": y_valid_minutes_enc
}


In [ ]:
def custom_mse(y_true, y_pred):
    """
    Deal with cyclical nature of time for minutes
    0-59 minutes, so difference between 0 and 59 is 1 minute
    """
    y_true = tf.cast(y_true, dtype=tf.float32)
    diff = tf.abs(y_true - y_pred)
    cyclical_diff = tf.minimum(diff, 60.0 - diff)
    return tf.reduce_mean(tf.square(cyclical_diff))

In [ ]:

inputs = keras.Input(shape=(pixel_size, pixel_size, 1), name="input_image")
x = keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D((2,2))(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(256, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512, activation="leaky_relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
shared_features = keras.layers.Dropout(0.5)(x)
hour_branch = keras.layers.Dense(64, activation="leaky_relu", kernel_regularizer=keras.regularizers.l2(0.001))(shared_features)
hour_branch = keras.layers.Dropout(0.5)(hour_branch)
hour_output = keras.layers.Dense(12, activation="softmax", name="hour_output")(hour_branch)
minute_branch = keras.layers.Dense(128, activation="leaky_relu",
                                   kernel_regularizer=keras.regularizers.l2(0.001))(shared_features)
minute_branch = keras.layers.Dropout(0.5)(minute_branch)
minute_output = keras.layers.Dense(1, activation="linear", name="minute_output")(minute_branch)

model = keras.Model(inputs=inputs, outputs=[hour_output, minute_output])
minute_loss_weight = 0.002 #mse is a lot higher than crossentropy. minutes also influence final time less.
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss={
        "hour_output": "categorical_crossentropy",
        "minute_output": "mean_squared_error"
        # custom_mse
    },
    loss_weights={
        "hour_output": 1,
        "minute_output": minute_loss_weight
    },
    metrics={
        "hour_output": "accuracy",
        "minute_output": 'mean_absolute_error'
    }
)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 150, 150,  │        320 │ input_image[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 150,  │        128 │ conv2d_21[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 75, 75,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 75, 75,    │          0 │ max_pooling2d_12… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 75, 75,    │     18,496 │ dropout_6[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 75, 75,    │        256 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 75, 75,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 75, 75,    │        256 │ conv2d_23[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 37, 37,    │          0 │ max_pooling2d_13… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 37, 37,    │     73,856 │ dropout_7[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        512 │ conv2d_24[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 37, 37,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        512 │ conv2d_25[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 18, 18,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 18, 18,    │          0 │ max_pooling2d_14

 Total params: 2,772,109 (10.57 MB)

 Trainable params: 2,770,765 (10.57 MB)

 Non-trainable params: 1,344 (5.25 KB)

In [ ]:
import keras.backend as K

lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.10,          # halce the learning rate if no improvement
    patience=5,          # Wait 4 epochs with no improvement before reducing
    min_lr=1e-7        # Set a minimum learning rate at 1e-6
)
early_stopper = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,          # Wait 8 epochs for improvement before stopping
    restore_best_weights=True  # Automatically restore the model weights from the best epoch
)
# model.fit(
#     X_train, y_train_formatted,
#     epochs=80,
#     validation_data=(X_valid, y_valid_formatted),
#     callbacks=[lr_scheduler, early_stopper]
#     )
# K.clear_session()

In [ ]:
def common_sense_loss(y_true, y_pred):
        y_true_hour = np.argmax(y_true["hour_output"], axis=1)
        y_true_minute = np.array(y_true["minute_output"], dtype=float)
        y_pred_hour = np.argmax(y_pred[0], axis=1)
        y_pred_minute = y_pred[1].squeeze()
        y_true_total_minutes = y_true_hour * 60 + y_true_minute
        y_pred_total_minutes = y_pred_hour * 60 + y_pred_minute
        diff = y_true_total_minutes - y_pred_total_minutes
        abs_diff = tf.abs(diff)


        cyclical_error = tf.minimum(abs_diff, 720.0 - abs_diff)

        return cyclical_error

In [ ]:
# y_test_minutes_enc = np.array(y_test_minutes, dtype=float)
# model.evaluate(X_test, {
#     "hour_output": y_test_hours_enc,
#     "minute_output": y_test_minutes_enc
# })
# y_test_formatted = {
#     "hour_output": y_test_hours_enc,
#     "minute_output": y_test_minutes_enc
# }
# y_pred_list = model.predict(X_test)

# np.mean((common_sense_loss(y_test_formatted, y_pred_list)))
# #7.683456
# #8.94813338364164
# (8.887957806802458)


regression esque

In [ ]:
#label prep for regression model
labels = np.load(labels_path)
labels_in_minutes = []
for label in labels:
    total_minutes = label[0] * 60 + label[1]
    labels_in_minutes.append(total_minutes)
labels_in_minutes = np.array(labels_in_minutes)
X_train_full, X_test,y_train_full, y_test = train_test_split(
    images, labels_in_minutes, test_size=0.1, random_state=35
)
X_train, X_valid,y_train, y_valid = train_test_split(
    X_train_full, y_train_full, test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default


In [ ]:
def create_regmodel():
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,          # halve the learning rate if there is no improvement
        patience=3,          # Wait 2 epochs with no improvement before reducing
        min_lr=1e-6          # Set a minimum learning rate at 1e-6
    )
    early_stopper = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=7,          # Wait 6 epochs for improvement before stopping
        restore_best_weights=True  # Automatically restore the weights from the best epoch
    )
    model= keras.models.Sequential([
        keras.Input(shape=(pixel_size, pixel_size, 1)),
        # Block 1
        keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Dropout(0.2),
        # Block 2
        keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.2),
        # Block 3
        keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.2),
        # Block 4
        keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, (3,3), activation="relu", padding="same"),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.2),


        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="leaky_relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="leaky_relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation="linear"),
    ])
    # cse = common_sense_loss
    model.compile(loss='MeanSquaredError',
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['mean_absolute_error',
            #   common_sense_loss
            #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
            ],
    )
    return model, lr_scheduler, early_stopper
model, lr_scheduler, early_stopper = create_regmodel()

In [ ]:
# model.fit(
#     X_train, y_train,
#     epochs=60,
#     validation_data=(X_valid, y_valid),
#     callbacks=[lr_scheduler, early_stopper]
#     )

In [ ]:
def common_sense_loss(y_true, y_pred):
        y_true = tf.cast(tf.squeeze(y_true), dtype=tf.float32)
        y_pred = tf.cast(tf.squeeze(y_pred), dtype=tf.float32)
        diff = y_true - y_pred
        abs_diff = tf.abs(diff)

        cyclical_error = tf.minimum(abs_diff, 720.0 - abs_diff)
        return float(tf.reduce_mean(cyclical_error))

In [ ]:
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
print(y_pred.shape, y_test.shape)
print(common_sense_loss(y_test, y_pred.squeeze()))

2025-11-02 13:27:38.374084: I external/local_xla/xla/service/service.cc:163] XLA service 0x76416400ee40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-02 13:27:38.374108: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-11-02 13:27:38.386019: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-02 13:27:38.450710: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91400
2025-11-02 13:27:38.460409: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-02 13:27:38.

22/57 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 163810.9531 - mean_absolute_error: 346.1021

I0000 00:00:1762086460.260101  533990 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 172201.2344 - mean_absolute_error: 360.3383
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
(1800, 1) (1800,)
182.8462371826172


In [32]:

labels = np.load(labels_path)
#Separate hours and minutes
hours = labels[:, 0]
minutes = labels[:, 1]

#Transform hours into sine and cosine components
hour_sin = np.sin(2 * np.pi * hours / 12.0)
hour_cos = np.cos(2 * np.pi * hours / 12.0)
y_hour = np.stack([hour_sin, hour_cos], axis=1)
# Do the same for minutes
minute_sin = np.sin(2 * np.pi * minutes / 60.0)
minute_cos = np.cos(2 * np.pi * minutes / 60.0)
y_minute = np.stack([minute_sin, minute_cos], axis=1)

# Split into train_full (90%) and test (10%)
X_train_full, X_test, \
y_hour_train_full, y_hour_test, \
y_minute_train_full, y_minute_test = train_test_split(
    images, y_hour, y_minute, test_size=0.1, random_state=35
)

X_train, X_valid, \
y_hour_train, y_hour_valid, \
y_minute_train, y_minute_valid = train_test_split(
    X_train_full, y_hour_train_full, y_minute_train_full,
    test_size=1/9, random_state=35
) # 1/9 x 0.9 = 0.1. train test split shuffles by default


In [ ]:
inputs = keras.Input(shape=(pixel_size, pixel_size, 1), name="input_image")
x = keras.layers.Conv2D(32, (3,3), activation="relu", padding="same")(inputs)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D((2,2))(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Dropout(0.15)(x)

x = keras.layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Dropout(0.15)(x)

# x = keras.layers.Conv2D(256, (3,3), activation="relu", padding="same")(x)
# x = keras.layers.BatchNormalization()(x)
# x = keras.layers.MaxPooling2D(2)(x)
# x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Flatten()(x)

x = keras.layers.Dense(128, activation="leaky_relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
shared_features = keras.layers.Dropout(0.3)(x)
hour_branch = keras.layers.Dense(64, activation="leaky_relu", kernel_regularizer=keras.regularizers.l2(0.001))(shared_features)
hour_branch = keras.layers.Dropout(0.3)(hour_branch)
hour_output = keras.layers.Dense(2, activation="tanh", name="hour_output")(hour_branch)
minute_branch = keras.layers.Dense(128, activation="leaky_relu",
                                   kernel_regularizer=keras.regularizers.l2(0.001))(shared_features)
minute_branch = keras.layers.Dropout(0.3)(minute_branch)
minute_output = keras.layers.Dense(2, activation="linear", name="minute_output")(minute_branch)

model = keras.Model(inputs=inputs, outputs=[hour_output, minute_output])
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss={
        "hour_output": "mean_squared_error",
        "minute_output": "mean_squared_error"
        # custom_mse
    },
    loss_weights={
        "hour_output": 1,
        "minute_output": 0.5
    },
    metrics={
        "hour_output": "mean_absolute_error",
        "minute_output": 'mean_absolute_error'
    }
)
model.summary()

Model: "functional_22"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_138 (Conv2D) │ (None, 150, 150,  │        320 │ input_image[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 150, 150,  │        128 │ conv2d_138[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_86    │ (None, 75, 75,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_96          │ (None, 75, 75,    │          0 │ max_pooling2d_86… │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_139 (Conv2D) │ (None, 75, 75,    │     18,496 │ dropout_96[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 75, 75,    │        256 │ conv2d_139[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_140 (Conv2D) │ (None, 75, 75,    │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 75, 75,    │        256 │ conv2d_140[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_87    │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_97          │ (None, 37, 37,    │          0 │ max_pooling2d_87… │
│ (Dropout)           │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_141 (Conv2D) │ (None, 37, 37,    │     73,856 │ dropout_97[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        512 │ conv2d_141[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_142 (Conv2D) │ (None, 37, 37,    │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 37, 37,    │        512 │ conv2d_142[0][0]  │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_88    │ (None, 18, 18,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_98          │ (None, 18, 18,    │          0 │ max_pooling2d_88

 Total params: 5,612,548 (21.41 MB)

 Trainable params: 5,611,716 (21.41 MB)

 Non-trainable params: 832 (3.25 KB)

In [ ]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,          # halve the learning rate if there is no improvement
        patience=3,          # Wait 2 epochs with no improvement before reducing
        min_lr=1e-6          # Set a minimum learning rate at 1e-6
    )
early_stopper = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=7,          # Wait 6 epochs for improvement before stopping
        restore_best_weights=True  # Automatically restore the weights from the best epoch
    )
model.fit(
    X_train, {"hour_output": y_hour_train, "minute_output": y_minute_train},
    epochs=80,
    validation_data=(X_valid, {"hour_output": y_hour_valid, "minute_output": y_minute_valid}),
    callbacks=[lr_scheduler, early_stopper]
    )


Epoch 1/80
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - hour_output_loss: 0.6042 - hour_output_mean_absolute_error: 0.6213 - loss: 1.4207 - minute_output_loss: 0.7040 - minute_output_mean_absolute_error: 0.6873 - val_hour_output_loss: 0.6088 - val_hour_output_mean_absolute_error: 0.6323 - val_loss: 1.3486 - val_minute_output_loss: 0.5451 - val_minute_output_mean_absolute_error: 0.6245 - learning_rate: 1.0000e-06
Epoch 2/80
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - hour_output_loss: 0.5614 - hour_output_mean_absolute_error: 0.6014 - loss: 1.3408 - minute_output_loss: 0.6305 - minute_output_mean_absolute_error: 0.6561 - val_hour_output_loss: 0.5899 - val_hour_output_mean_absolute_error: 0.6235 - val_loss: 1.3365 - val_minute_output_loss: 0.5595 - val_minute_output_mean_absolute_error: 0.6324 - learning_rate: 1.0000e-06
Epoch 3/80
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - hour_output_loss: 0.5397 - hour_output_mean_absolute_error: 0.5907 - loss: 1.3123 - minute_output_loss: 0.6176 -

In [28]:
def sin_cos_to_time(sin_cos,period):
    sin_component = sin_cos[:, 0]
    cos_component = sin_cos[:, 1]
    angles = np.arctan2(sin_component, cos_component)
    angle_positive = np.mod(angles, 2 * np.pi)
    time_values = (angle_positive / (2 * np.pi)) * period
    return np.round(time_values).astype(int)


In [29]:
#save the trained model
# model.save("time_prediction_model_dual_head_2.keras")
model = keras.models.load_model("time_prediction_model_dual_head.keras")

In [33]:
y_pred = model.predict(X_test)
pred_hours = sin_cos_to_time(y_pred[0], 12)
pred_minutes = sin_cos_to_time(y_pred[1], 60)
total_pred_minutes = pred_hours * 60 + pred_minutes

#do the same for true values
true_hours = sin_cos_to_time(y_hour_test, 12)
true_minutes = sin_cos_to_time(y_minute_test, 60)
total_true_minutes = true_hours * 60 + true_minutes
error_in_minutes = total_true_minutes - total_pred_minutes
abs_error = np.abs(error_in_minutes)
cyclical_error = np.minimum(abs_error, 720 - abs_error)

print("Mean absolute error in minutes:", np.mean(cyclical_error))


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Mean absolute error in minutes: 4.918888888888889


In [ ]:
#sanity check
for i in range(1000):
    if cyclical_error[i] > 20:
        print(f"Predicted time: {pred_hours[i]:02d}:{pred_minutes[i]:02d} | True time: {true_hours[i]:02d}:{true_minutes[i]:02d} | Error in minutes: {cyclical_error[i]}")

Predicted time: 04:17 | True time: 05:00 | Error in minutes: 43
Predicted time: 00:23 | True time: 10:01 | Error in minutes: 142
Predicted time: 03:21 | True time: 01:54 | Error in minutes: 87
Predicted time: 04:23 | True time: 05:05 | Error in minutes: 42
Predicted time: 06:28 | True time: 10:40 | Error in minutes: 252
Predicted time: 05:25 | True time: 06:17 | Error in minutes: 52
Predicted time: 01:13 | True time: 03:57 | Error in minutes: 164
Predicted time: 03:24 | True time: 04:51 | Error in minutes: 87
Predicted time: 09:55 | True time: 11:41 | Error in minutes: 106
Predicted time: 02:11 | True time: 01:06 | Error in minutes: 65
Predicted time: 05:19 | True time: 00:05 | Error in minutes: 314
Predicted time: 07:40 | True time: 06:35 | Error in minutes: 65
Predicted time: 03:14 | True time: 07:03 | Error in minutes: 229
Predicted time: 02:13 | True time: 00:02 | Error in minutes: 131
Predicted time: 03:28 | True time: 09:22 | Error in minutes: 354
Predicted time: 05:32 | True tim